## Deep Learning to Predict Species, Gene Ontology Terms with Promoter Sequences

### Project Summary

**Background:**  
Studies have shown that there is a high degree of similarity in the DNA sequences that compose orthologous gene body regions from different species. However, the regulatory regions that exist between these gene bodies have been shown to be notably different. These differences are thought to be the key drivers in the variability in gene expression levels between species for orthologous genes. Because the differences in these regulatory elements are associated with differences in the binding affinity of related transcription factors, these regulatory elements likely possess interesting features that may enable us to better understand the underlying differences between species and how they have evolved. In addition, these regulatory regions are different from gene to gene, and it appears that genes with regulatory elements similar to each other are often associated with the same functionality or ontology. Thus, we may be able to exploit underlying differences in the DNA sequences of gene promoter regions to differentiate or predict species as well as gene ontology.

**Goals:**  
1) Use deep representational learning to predict species from promoter sequences  
2) Use deep representational learning to predict gene ontology terms for each gene from the gene's promoter sequence

### Data

**Promoter Sequences:** GFF files, FASTA files from Ensembl (check different kingdom sections, i.e. EnsemblFungi)  
*S. cerevisiae*  
*C. elegans*   
*S. pombe*  
*C. albicans*  

NOTE: FASTA files downloaded contained masked DNA sequenced (i.e. "dna_rm" in file name when FTP from Ensembl)  
NOTE: promoter sequences manually curated by integrating annotations of GFF files and sequences in FASTA files  
NOTE: sequences 1000bp upstream of the start of the coding region have been personally written to .fa files (can be found under /my_promoters directory)  

**Gene Ontology:** AmiGO2 (http://amigo.geneontology.org/)  
*S. cerevisiae* (contributor: SGD)  

Custom download: Source (source), Organism (taxon_subset_closure_label), Gene/product (bioentity_label), 

GO Terms: cell cycle, chromosome organization, response to stress, cellular localization, kinase activity, pyrophosphatase activity  

**KEGG Pathways:** Broad Institute (http://software.broadinstitute.org/gsea/msigdb/collections.jsp#C2)  

**Old Promoter Sequences:** Eukaryotic Promoter Database (epd.vital-it.ch) - *IGNORE*    
*S. cerevisiae* (database: S_cerevisiae_epdnew, assembly: sacCer3) - 5117 promoters (when including all selection parameters), file: sCer.fa.txt  
*S. pombe* (database: S_pombe_epdnew, assembly: spo2) - 3440 promoters (when including all selection parameters), file: sPom.fa.txt  
*C. elegans* (database: worm_epdnew, assembly: ce6) - 7120 promoters (when including all selection parameters), file: cEleg.fa.txt  
*H. sapiens* (database: human_epdnew, assembly: hg19) - 25503 promoters (when including all selection parameters), file: human.fa.txt  

### Species Prediction
**Approach:**  
1) build CNN (2 convolutional layers for now)  
2) use representational layer to differentiate species (visualize using PCA, tSNE plots)  
3) extract features from DNA sequences that are most predictive of species (possibly via clustering - identify similar features between clustered groups; also use clustering to visualize separation of species)  

**Important Details:**  
Random sampling: In order to get balanced numbers of training examples for each species, we must randomly sample sequences from the species datasets with larger numbers of sequences. We can do this in two different ways due to the sliding window approach implemented.  
1) At first, we simply randomly sampled with respect to genes such that, for each species, the same number of genes/promoter regions were represented. In this case, the same number of sequences would be used for each species, as the number of windows generated for each gene/promoter region was chosen to be the same across all genes.  
2) In addition, we can also randomly sample from the entire set of windows created across all genes. If this approach is taken, then a larger set of genes would be accounted for in the training/validation data but fewer examples for each gene/promoter region would appear.  

When using the Python script that utilizes HDF5 formatted sequence data files, the latter approach must be used, as the data stored in the HDF5 files are simply the one-hot encoded sequences for all windows generated via the sliding window approach across all genes.  

Also, is it okay to use unbalanced numbers of training examples from each species as long as each batch submitted for training is the same???  

**Sampling Issues:**  
As mentioned above, the way in which we sample "windows" of DNA sequences from the promoter regions of genes can have a big impact on the way the model is trained and, ultimately, the performance of the model when testing. In particular, we've realized that there is a potential problem with the first method of sampling outlined above in which a certain set of genes is sampled for each species and all of the windows created for its corresponding promoter regions are used to train and validate. As of now, we've been getting very high accuracy measures (> 0.95) when using this approach, as there is likely to be a fair degree of overlap in genes represented in the training data and the validation data, although the actual windows in the genes are not the same. Because of the way convolutional neural networks function, patterns recognized by the convolutional filters are not space-dependent, meaning that as long as the specific pattern appears in an inputted DNA sequence, the network will be able to properly classify the associated labels.  

This is a problem in our case for two main reasons. Firstly, we would basically be testing our trained model on data that is very similar to our training data because of the overlap in DNA sequence patterns across different windows of the same DNA region. Secondly, the model is only being trained on a small subset of these DNA sequences, so the patterns (or potentially "motifs") that are being identified are limited to their respective genes. Genes that are affected by transcription factors that are not represented in the training data will not be applicable to our model. Thus, not only do we face the possibility of "overfitting" to the set of genes represented in the data set, we lose generalizability due to the lack of diversity in genes/their promoter regions.  

Related to the sampling method as well is the diversity of genes represented per species. As of now, our training data ensures that there is an equal number of promoter regions/genes represented for each species. For species with larger number of genes, we therefore diminish the number of examples that are to be used for training to meet this requirement. Thus, for genomes that are substantially larger than the smallest genome in the dataset, we are losing a very large number of training examples. We, therefore, need to modify this approach...  

**Sampling Issues: (continued)** - 7/14/17  
Even when trying the above approach, we observe inaccurate predictions in a few notable cases. The most distinct one is the case in which we predict between S. cerevisiae, S. pombe, mouse, and human promoters. Even though projections of the representational layers indicate greater "separation" between S. cerevisiae and S. pombe, we observed greater confusion in predicting between these species than in predicting between mouse and human promoters, which are very clearly mixed with each other in the projection visualizations. I currently suspect that this observation may be due to the high number of "windows" that we've used thus far for each gene/promoter, especially for species with lower gene counts like S. cerevisiae or S. pombe. In the previous approach, we ensured that all genes were represented and that we maintained an equal number of promoter "windows" for each species, but in doing so, we generate many more "windows" per gene for S. cerevisiae and S. pombe. As a consequence, the model may be overfitting promoters for these species and, thus, resulting in "confusion" between these species (i.e. the model becomes less generalizable).  

In our updated approach, we simply set a threshold for the number of windows that are to be randomly sampled from the set of windows generated. This threshold has been set to 15.  

**Confusion Matrix (initial sampling method):** 1) S. cerevisiae, 2) C. elegans, 3) Mouse, 4) Human  

All Genes Test Set (Accuracy: ~0.7)

In [6]:
np.array([[ 7907,  4271,   121,   312],
       [ 6329,  7871,    70,   308],
       [  182,   100, 14342,  1409],
       [   93,    52,   495,  4049]])

array([[ 7907,  4271,   121,   312],
       [ 6329,  7871,    70,   308],
       [  182,   100, 14342,  1409],
       [   93,    52,   495,  4049]])

Validation Set (Accuracy: >0.95)  

0.7131765147878357

### Feature Importance Via Gradients

In order to determine the importance of features in classifying species based on DNA sequences, we consider the components of inputted DNA sequences that are critical in enabling the classifier to make its predictions. We do so using a method demonstrated in the following link: http://www.unofficialgoogledatascience.com/2017/03/attributing-deep-networks-prediction-to.html.  

In this method, feature importance is determined by calculating the gradients of the output (species prediction scores) with respect to our input (one-hot encoded DNA sequences). When we multiply the inputted one-hot encoded DNA sequence by the corresponding gradient, we are in effect using a first-order Taylor approximation of the maximum prediction score (the largest output of the Softmax function in the last layer, i.e. our predicted label). By performing this element-wise multiplication, we expect to "highlight" the portions of the inputted DNA sequence that contribute the most greatly to the predicted label. However, as is the case with classifying images in the example from the online link, a straightforward gradient calculation and subsequent Taylor series approximation is insufficient for accurately approximating the output. When determining the gradients associated with a randomly chosen DNA sequence, we see immediately that almost all of the gradient values to be multiplied to the input are very low (~10^-5). This phenomenon is due largely in part to the rapidity of saturation for the label-specific prediction score near the input. In other words, when scaling down the "intensities" of the inputs via a scaling parameter, we would observe that the prediction scores reach their maximum values at low values of the scaling parameter.  

In order to combat this obstacle, we can use integrated gradients, as is done in the online example. 

### Gene Ontology Prediction
**Approach:**   
1) build CNN (2 convolutional layers for now)  
2) use representational layer to differentiate species  

**Convolutional Neural Network Details**  
- *Multi-label classification:* apply sigmoid function in the last fully-connected layer instead of SoftMax (Softmax generates outputs that can be interpreted as categorical distributions, but gene set labels are not mutually exclusive so the sigmoid function is more appropriate.)  
- Accuracy: **Hamming loss**  (fraction of wrong labels to the number of total labels), exact match percentage (simply involves rounding predictions at a threshold of 0.5 and identifying if rounded prediction labels match actual labels)  
- Loss function: **binary/multinomial cross entropy** (used instead of categorical cross entropy)  

### Analyzing Trained Model Parameters

After training the networks, we seek to understand the key features that enable us to differentiate between species and ontologies. We do this by taking a closer look at the layer that exists right before the final output layer. In our networks, this layer is a fully-connected layer that can be interpreted as our "representational layer". It possesses the same number of rows as the number of examples passed through the network and the number of columns is specified by the user. We can perform PCA and/or tSNE on this representational layer to better understand which features are important for segregating the different species in the predictive model.  

### Current Workflow

**Species Prediction**  
- GTF and FASTA files (Ensembl)  
- get and write promoter sequences to .fa files (see writeSeq.py)  
- create windows and write training and validation sets to H5 files (see writeHDF5.py)  
- train CNN model and save trained model (see main_h5.py)  
- get representational layers (see analysis.py)  


### Gene Ontology Annotations  

Due to the lack of success we've observed so far in predicting GO terms that encompass a large number of genes (+800 genes in S. cerevisiae), we seek to look into GO terms that appear further down in the GO hierarchical structure. We can use the DAG visualization tool here to identify the child nodes and node-to-node relations: http://www.ebi.ac.uk/QuickGO/.  

7/10/17  
Here, we first attempt GO term classification for child nodes of GO:0034599 (cellular response to oxidative stress) and focus specifically on child nodes that have a "part of" relationship to this node. These child nodes include the following terms.  
- oxidative stress-induced premature senescence (GO:0090403)  
- age-dependent response to oxidative stress involved in replicative cell aging (GO:0001322)  
- positive regulation of transcription factor import into nucleus in response to oxidative stress (GO:0036283)  
- cell death in response to oxidative stress (GO:0036473)  
- regulation of translation in response to oxidative stress (GO:0043556)  
- regulation of transcription from RNA polymerase II promoter in response to oxidative stress (GO:0043619).  

7/11/17 (IGNORE the terms from the previous day)  
The terms described above are too limited to perform a proper classification training task with our deep learning models, so we look to the parent node of the above node - response to stress (GO:0006950). While there are a number of child nodes associated with this GO term, we focus initially on the following:  
- response to oxidative stress  
- response to hypoxia  
- response to endoplasmic reticulum stress  
- cellular response to starvation  

NOTE: The terms above were determined by first obtaining the GO ID number of "response to stress" via EBI and then searching for this term with AMIGO, where we retain information about the species name, gene name, and GO class (direct)(annotation_class_label).  

7/12/17  
After learning about sequence-level features that differ between promoters, we focus more specifically on being able to categorize promoters based on the major promoter classes. Here, we take a closer look at classifying promoters of genes that belong to the following GO terms:  

**Type I promoter:** (tissue-specific)  
- skeletal system development (GO:0001501)  

**Type II promoter:** (ubiquitously expressed)  
- ribosome biogenesis (GO:0042254)  
- translation (GO:0006412)  

**Type III promoter:** (developmental genes)  
- multicellular organism development (GO:0007275)  
- cell differentiation (GO:0030154)  

**TCT promoter:** (translation machinery)  
- ribosome biogenesis (GO:0042254)  
- translation (GO:0006412)  

Each of these GO terms correspond to a distinct promoter class with unique sequence-level features separating them. We focus primarily on mouse promoters for now. (There are a lot of low quality reads for promoters associated with "ribosome biogenesis" for some reason...We may try to get different sequence data for humans in the future.)  

In [2]:
from sklearn.mixture import GaussianMixture
d = np.loadtxt('results/TensorflowProjector/sCer_sPom_rep.txt',delimiter='\t')
g = GaussianMixture(n_components=2)

In [3]:
g = g.fit(d)


In [4]:
plot_results(d,g.predict(d),g.means_,g.covariances_,0,'Gaussian Mixture')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in double_scalars
